<a href="https://colab.research.google.com/github/qaisazhar/Capstone-project-deteksi-katarak/blob/main/Capstone_Project_(deteksi_mata_katarak).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##import library

In [ ]:
import numpy as np
import cv2
import os
import matplotlib as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

##load dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.listdir("/content/drive/MyDrive/cataract_dataset_binary")

['2_cataract', '1_normal']

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

input_folder = "/content/drive/MyDrive/cataract_dataset_binary"

splitfolders.ratio(
    input_folder,
    output="/content/drive/MyDrive/cataract_dataset_split",
    seed=42,
    ratio=(0.7, 0.2, 0.1)
)


Copying files: 400 files [03:03,  2.18 files/s]


In [ ]:
train_dir = "/content/drive/MyDrive/cataract_dataset_split/train"
val_dir = "/content/drive/MyDrive/cataract_dataset_split/val"
test_dir = "/content/drive/MyDrive/cataract_dataset_split/test"

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)


Found 280 files belonging to 2 classes.
Found 80 files belonging to 2 classes.
Found 40 files belonging to 2 classes.


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))


Found 280 files belonging to 2 classes.


In [ ]:
import os

normal_count = len(os.listdir("/content/drive/MyDrive/cataract_dataset_split/train/1_normal"))
cataract_count = len(os.listdir("/content/drive/MyDrive/cataract_dataset_split/train/2_cataract"))

print("Normal:", normal_count)
print("Cataract:", cataract_count)


Normal: 210
Cataract: 70


##normalisasi

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

##bangun model dengan MobileNetV2

In [ ]:
from pickle import FALSE
#load base model mobilenetv2
base_model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

#tambah lapisan custom
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x) #binary classification

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique([0,1]),
    y=[0]*210 + [1]*70 #0=normal, 1=cataract
)
class_weights = dict(enumerate(class_weights))
print(class_weights)

{0: np.float64(0.6666666666666666), 1: np.float64(2.0)}


##training model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 48s 5s/step - accuracy: 0.3109 - loss: 1.2751 - val_accuracy: 0.6875 - val_loss: 0.6208
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.3327 - loss: 1.2526 - val_accuracy: 0.7125 - val_loss: 0.6145
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.3645 - loss: 1.2474 - val_accuracy: 0.7125 - val_loss: 0.6083
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - accuracy: 0.3188 - loss: 1.2663 - val_accuracy: 0.7000 - val_loss: 0.6023
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 44s 5s/step - accuracy: 0.3434 - loss: 1.2025 - val_accuracy: 0.6875 - val_loss: 0.5970
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.3694 - loss: 1.1698 - val_accuracy: 0.7000 - val_loss: 0.5921
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.4017 - loss: 1.1404 - val_accuracy: 0.7000 - val_loss: 0.5878
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 46s 5s/step - accuracy: 0.3756 - loss: 1.1276 - val_accuracy: 0.7250 - val_loss: 0.5840
Epoch 9/

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print('Test accuracy:', test_acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 294ms/step - accuracy: 0.7500 - loss: 0.5755
Test accuracy: 0.75


In [ ]:
# Training tahap 2 (fine-tuning)
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights
)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.3907 - loss: 0.9482 - val_accuracy: 0.7375 - val_loss: 0.5751
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.3727 - loss: 0.8961 - val_accuracy: 0.7375 - val_loss: 0.5737
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.4044 - loss: 0.9244 - val_accuracy: 0.7375 - val_loss: 0.5728
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 43s 5s/step - accuracy: 0.3548 - loss: 0.9745 - val_accuracy: 0.7375 - val_loss: 0.5719
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.3527 - loss: 0.9332 - val_accuracy: 0.7375 - val_loss: 0.5710
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.4297 - loss: 0.8394 - val_accuracy: 0.7375 - val_loss: 0.5701
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.3262 - loss: 0.9207 - val_accuracy: 0.7375 - val_loss: 0.5693
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.3551 - loss: 0.9747 - val_accuracy: 0.7375 - val_loss: 0.5685
Epoch 9/